In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import notebook
import scipy.stats
from sklearn.metrics import mean_squared_error
%matplotlib inline
import numpy as np

import umap
import umap.plot
reducer = umap.UMAP(random_state=42)

plt.rcParams.update({'font.family': 'serif', 'font.serif': 'Arial', "figure.facecolor": 
                     'white', "axes.facecolor": 'white',
                     "savefig.facecolor": 'white'})

In [2]:
counts_file = 'data/Pancreatic/Pancreatic_RNAseq_tpm.csv'
counts_df = pd.read_csv(counts_file, index_col = 0)

# remove genes with constant values
constant_columns = counts_df.columns[counts_df.nunique() <= 1]
counts_df = counts_df.drop(columns = constant_columns)
counts_df.shape

(55, 50651)

In [3]:
viability_file = 'data/Pancreatic/viability.csv'
viability_df = pd.read_csv(viability_file, index_col = 0)
drugs = viability_df.columns
viability_df = (viability_df - viability_df.mean()) / viability_df.std()

shared_patients = list(set(counts_df.index) & set(viability_df.index))
print(len(shared_patients))
counts_df = counts_df.loc[shared_patients]
viability_df = viability_df.loc[shared_patients]

43


In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer

features_df = counts_df

ss = StandardScaler()
pt = PowerTransformer()

# OPTION 0 (no norm)
#features_norm_df = features_df

#OPTION 1
#features_norm_df = pd.DataFrame(pt.fit_transform(features_df), index = features_df.index, columns = features_df.columns)

# OPTION 2
#features_norm_df = pd.DataFrame(ss.fit_transform(features_df), index = features_df.index, columns = features_df.columns)

# OPTION 3 (Seurat normalization)
#features_norm_df = np.log2((features_df.T / features_df.T.sum().values) * 10000 + 1).T 

# OPTION 4
def _handle_zeros_in_scale(scale, copy=True):
    """
    This method is copied `from sklearn.preprocessing._data`
    Makes sure that whenever scale is zero, we handle it correctly.
    This happens in most scalers when we have constant features.
    """
    # if we are fitting on 1D arrays, scale might be a scalar
    if np.isscalar(scale):
        if scale == 0.0:
            scale = 1.0
        return scale
    elif isinstance(scale, np.ndarray):
        if copy:
            # New array to avoid side-effects
            scale = scale.copy()
        scale[scale == 0.0] = 1.0
        return scale

def transform_standardize(data, mean, std):
    return (data - mean) / _handle_zeros_in_scale(std, copy=False)

features_norm_df = transform_standardize(features_df, features_df.mean(), features_df.std().values)

constant_columns = features_norm_df.columns[features_norm_df.nunique() <= 1]
features_norm_df = features_norm_df.drop(columns = constant_columns)
print('Removed constant features', len(constant_columns))

Removed constant features 1000


In [5]:
import decoupler as dc
dorothea = dc.get_dorothea()
progeny = dc.get_progeny()
dorothea_genes = list(set(dorothea.source.unique()) & set(dorothea.target.unique()))
progeny_genes = list(set(progeny.target.unique()))
net_genes = list(set(dorothea_genes+progeny_genes))
shared_genes = list(set(net_genes) & set(features_norm_df.columns))

In [6]:
feat_dist = 1-features_norm_df.corr().abs()

In [8]:
from sklearn.cluster import AgglomerativeClustering

gene_clusters = AgglomerativeClustering(affinity = 'precomputed', 
                                        n_clusters = None, linkage = 'complete',
                                       distance_threshold = 0.4).fit(feat_dist)
print(len(feat_dist), len(np.unique(gene_clusters.labels_)))

49651 17387


In [40]:
gene_clusters_df = pd.DataFrame({'cluster': gene_clusters.labels_}, index = feat_dist.index)

In [47]:
def select_genes(gene_clusters_df):
    selected_genes = []
    for c, rows in gene_clusters_df.groupby('cluster'):
        genes_in_net = rows[rows.index.isin(net_genes)]
        if len(genes_in_net)==0: sel_gene = rows.index.values[0]
        else: 
            sel_gene = genes_in_net.index.values[0]
        selected_genes.append(sel_gene)
    return selected_genes

stop = False
while not stop:
    print('Next iteration')
    selected_genes = select_genes(gene_clusters_df)
    
    gene_clusters_loc = AgglomerativeClustering(affinity = 'precomputed', 
                                    n_clusters = None, linkage = 'complete',
                                    distance_threshold = 0.4).fit(feat_dist.loc[selected_genes][selected_genes])
    gene_clusters_df = pd.DataFrame({'cluster': gene_clusters_loc.labels_}, index = selected_genes)
    if len(np.unique(gene_clusters_loc.labels_)) == len(selected_genes):
        stop = True  
        
    print(len(selected_genes), len(np.unique(gene_clusters_loc.labels_)))

Next iteration
17387 10622
Next iteration
10622 8039
Next iteration
8039 6941
Next iteration
6941 6464
Next iteration
6464 6222
Next iteration
6222 6099
Next iteration
6099 6042
Next iteration
6042 6009
Next iteration
6009 5990
Next iteration
5990 5975
Next iteration
5975 5967
Next iteration
5967 5965
Next iteration
5965 5964
Next iteration
5964 5963
Next iteration
5963 5963


In [53]:
test = feat_dist.loc[selected_genes][selected_genes]
test.where(test.gt(0)).min(0).min()

0.40000941575954496

In [1]:
test.where(test.gt(0)).min(0).sort_values()

NameError: name 'test' is not defined

In [56]:
import pickle
with open('data/Pancreatic/non_colin_genes', 'wb') as fp:
    pickle.dump(selected_genes, fp)